# Introduction
In this small task, **clustering KMeans** using **big data** tool (e. g., **pyspark**) is deployed. The data for exploration and classification is derived from the very popular source. [data from kddcup99](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html).

- In the first step, the data with 41 attributes are read. Most of the data are double type except 3 attributes are String type. **SparkContext** method are used to read and parse each line into 41 attributes under **Resilient Distributed Dataset**. Then data is converted to again dataframe due to the nominal categorical features that the algorithms can not understand. 
- The **Pipeline process** has been deploy to reduce the manually repeated the same process over time. Indeed, there are many categorical features need to be indexed and converted to numerical format, and **VectorAssembler** is used to combine all necessary features as a single feature to be inputted into classifier.
- The **anomaly dection** means that the data or observations that are different from normal or what we expected. By doing that, statistic method flags the data as 'bad' or 'malicious' when the data behave five times different from the mean of all training data. This method acts on univariate outliers, it is easy to do but it is not really robust because it required the parameters about the distribution, like mean and standard deviation, and it also assumes that the model is normal. In addition, To obtain the **mean** and the **standard deviation**, the **Stats()** funtion is acted on the RDD. However, univariate outliers are not efficient in reality because, multiple actions are often considered to access normal or obnormal. Then, the multivariate should be used, for the sake of the simplicity we do not mention it in this small task.
- Instead, the **Kmeans** is used to visualize the outliers. To verify the dependence of the Kmeans on the data. There are two steps have been done to compare how sensitive KMeans to data. In the first case, the data is not standardized, mean that just use the data with different distribution for all features and let KMeans find the outliers. In the other case, all the features are standardized to ensure that they have the same distribution, indded all the features data are subtracted for their mean and divided by their standard deviation.
- The results show that, for the data without standardize, the KMeans recognizes so many outliers, more than 1000 in cluser 0 and cluster 2. However, when standardizing the data according to all input features, the KMeans gave us only one outliers in cluster 2.  

## 1. Setup google colab environment for pyspark and hadoop

In [0]:
'''
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
'''
from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/pyspark/spark-2.4.5-bin-hadoop2.7.tgz' .

!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


##### SparkContext is used to read data in RDD dataset.
- Then data is processed, all the data is float type.
- There are 3 features are string type. 

In [0]:
# load library
import numpy as np
from pyspark import  SparkContext
from pyspark.sql import SQLContext

sc = SparkContext.getOrCreate()
sqlcontex = SQLContext(sc)

# read data and convert label into numerical data
def schema_line(line):
  value = []
  for idx in range(len(line)):
    if idx >=1 and idx <=3:
      value.append(str(line[idx]))
    else:
      value.append(float(line[idx]))  
  return value
url = '/content/gdrive/My Drive/pyspark/kdd'
df = (sc.textFile(url).map(lambda x: x.split(','))
                      .map(schema_line)
                      .toDF())
df.show()
df.printSchema()

+---+---+--------+---+------+-------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+---+---+---+----+-----+-----+----+----+----+----+----+---+----+----+
| _1| _2|      _3| _4|    _5|     _6| _7| _8| _9|_10|_11|_12|_13|_14|_15|_16|_17|_18|_19|_20|_21|_22| _23| _24|_25|_26|_27|_28|_29|_30| _31|  _32|  _33| _34| _35| _36| _37| _38|_39| _40| _41|
+---+---+--------+---+------+-------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+---+---+---+----+-----+-----+----+----+----+----+----+---+----+----+
|0.0|udp| private| SF| 105.0|  146.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 1.0| 1.0|0.0|0.0|0.0|0.0|1.0|0.0| 0.0|255.0|254.0| 1.0|0.01| 0.0| 0.0| 0.0|0.0| 0.0| 0.0|
|0.0|udp| private| SF| 105.0|  146.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 1.0| 1.0|0.0|0.0|0.0|0.0|1.0|0.0| 0.0|255.0|254.0| 1.0|0.01| 0.0| 0.0| 0.0|0.0| 0.0| 0.0|
|0.0|udp| private| SF| 105.0|  146.0|0.0

#### Below code shows us the unique categorical for 3 string features

In [0]:
# check categorical value of column 1, 2 and 3
print('categorical 1: ', sc.textFile(url).map(lambda x: x.split(',')[1]).distinct().collect())
print('categorical 2: ', sc.textFile(url).map(lambda x: x.split(',')[2]).distinct().collect())
print('categorical 3: ', sc.textFile(url).map(lambda x: x.split(',')[3]).distinct().collect())


['tcp', 'udp', 'icmp']

## 2. Pipeline process

In [0]:
# load necessary libraries for project
from pyspark.ml.feature import StringIndexer 
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# pipeline
stages = []

# onehot 
columns = ['_2', '_3', '_4']
for column in columns:
  stringindexer = StringIndexer(inputCol = column, outputCol = column + 'index')
  onehot = OneHotEncoderEstimator(inputCols=[stringindexer.getOutputCol()], outputCols=[stringindexer.getOutputCol() + 'onehot'], dropLast=False)
  stages += [stringindexer, onehot]
# vector assembler
inputcol = [item for item in (df.columns + [c + 'indexonehot' for c in columns]) if item not in columns]  
vectorassembler = VectorAssembler(inputCols = inputcol , outputCol = 'features')
stages += [vectorassembler]

# pipeline model
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
model = pipelineModel.transform(df)


In [0]:
# show the results acting on the dataframe after pipeline
# result now stored in model (dataframe)
model.show(10)
model.rdd.map(lambda x: x.features).take(10)

+---+---+--------+---+-----+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+----+----+----+----+---+---+---+---+-------+-------------+-------+--------------+-------+--------------+--------------------+
| _1| _2|      _3| _4|   _5|   _6| _7| _8| _9|_10|_11|_12|_13|_14|_15|_16|_17|_18|_19|_20|_21|_22|_23|_24|_25|_26|_27|_28|_29|_30|_31|  _32|  _33| _34| _35| _36| _37|_38|_39|_40|_41|_2index|_2indexonehot|_3index| _3indexonehot|_4index| _4indexonehot|            features|
+---+---+--------+---+-----+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+----+----+----+----+---+---+---+---+-------+-------------+-------+--------------+-------+--------------+--------------------+
|0.0|udp| private| SF|105.0|146.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|1.0|1.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|255.0|254.0| 1.0|0.01| 0.0| 0.0|0.0|0.0|0.0|0.0|  

[SparseVector(117, {1: 105.0, 2: 146.0, 19: 1.0, 20: 1.0, 25: 1.0, 28: 255.0, 29: 254.0, 30: 1.0, 31: 0.01, 40: 1.0, 42: 1.0, 106: 1.0}),
 SparseVector(117, {1: 105.0, 2: 146.0, 19: 1.0, 20: 1.0, 25: 1.0, 28: 255.0, 29: 254.0, 30: 1.0, 31: 0.01, 40: 1.0, 42: 1.0, 106: 1.0}),
 SparseVector(117, {1: 105.0, 2: 146.0, 19: 1.0, 20: 1.0, 25: 1.0, 28: 255.0, 29: 254.0, 30: 1.0, 31: 0.01, 40: 1.0, 42: 1.0, 106: 1.0}),
 SparseVector(117, {1: 105.0, 2: 146.0, 19: 2.0, 20: 2.0, 25: 1.0, 28: 255.0, 29: 254.0, 30: 1.0, 31: 0.01, 40: 1.0, 42: 1.0, 106: 1.0}),
 SparseVector(117, {1: 105.0, 2: 146.0, 19: 2.0, 20: 2.0, 25: 1.0, 28: 255.0, 29: 254.0, 30: 1.0, 31: 0.01, 32: 0.01, 40: 1.0, 42: 1.0, 106: 1.0}),
 SparseVector(117, {1: 105.0, 2: 146.0, 19: 2.0, 20: 2.0, 25: 1.0, 28: 255.0, 29: 255.0, 30: 1.0, 32: 0.01, 40: 1.0, 42: 1.0, 106: 1.0}),
 SparseVector(117, {1: 29.0, 19: 2.0, 20: 1.0, 25: 0.5, 26: 1.0, 28: 10.0, 29: 3.0, 30: 0.3, 31: 0.3, 32: 0.3, 40: 1.0, 46: 1.0, 106: 1.0}),
 SparseVector(117, {1

In [0]:
# convert model (dataframe) into data (rdd)
# load some libraries needed for converting
from pyspark.ml.linalg import DenseVector
import numpy as np
data = model.rdd.map(lambda x: np.array(DenseVector(x.features)))

# show first two lines of coverted rdd named 'data'
data.take(2)

[array([0.00e+00, 1.05e+02, 1.46e+02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 2.55e+02, 2.54e+02,
        1.00e+00, 1.00e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
        1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 

## 3. Anamoly detection using statistic

In [0]:
# anamoly detection using statistic model
# find anomaly data using univariate model
# the anamoly is detected if they are much 
# deviated from the mean. (e.g., more than five times) 
stats = data.map(lambda x: x[1]).stats()
mean, std = stats.mean(), stats.stdev()
outliers = data.filter(lambda x: not(mean - 5*std < x[1] < mean + 5*std))
outliers.count()

7

## 4. Anamoly detection using Kmeans
#### 4.1 without nomalize the features

In [0]:
# Kmeans is more reliable than statistic method
# using Kmeans instead of statistics analytics
from pyspark.mllib.clustering import KMeans
clusters = KMeans.train(data, 5, maxIterations = 10, runs = 1, initializationMode = 'random')
clusters_size = data.map(lambda e: clusters.predict(e)).countByValue()
clusters_size

defaultdict(int, {0: 403, 1: 1656, 2: 544, 3: 308426})

In [0]:
# get the distance for each cluster in the assgined cluster
# and, show the histogram of all distances to see the oulier
def get_distance(clusters):
  def get_distance_map(record):
    cluster = clusters.predict(record)
    centroid = clusters.centers[cluster]
    dist = np.linalg.norm(record - centroid)
    return (dist, record) 
  return get_distance_map
distance_data = data.map(get_distance(clusters))
print('distance: \n', distance_data.take(10))
distance_data.keys().histogram(10)

distance: 
 [(671.8376236360743, array([0.00e+00, 1.05e+02, 1.46e+02, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 1.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 2.55e+02, 2.54e+02,
       1.00e+00, 1.00e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
       1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.

([301.837802875452,
  6234271.931457144,
  12468242.025111413,
  18702212.118765682,
  24936182.21241995,
  31170152.306074217,
  37404122.399728484,
  43638092.49338275,
  49872062.58703702,
  56106032.68069129,
  62340002.774345554],
 [311027, 0, 0, 0, 1, 0, 0, 0, 0, 1])

#### 4.2 Normalize all features by Standardizing all features

In [0]:
# The Kmeans is really sensitive the the data with high variance,
# it is a good practice to normalize all features before apply Kmeans
# standardize dataset (subtract the mean and devide for the stadard deviation)
# mean and stdev to standardize the rdd data
stats = data.map(lambda x: x).stats()
mean, stdev = stats.mean(), stats.stdev()

# standardize function
def standardize(line):
  return np.array([(line[i]-mean[i])/stdev[i] if abs(stdev[i]) > 1.0e-20 else line[i] for i in range(len(stdev))])

# show the data after standardizing
data_std = data.map(standardize)
data_std.take(10)

[array([-4.39176021e-02, -1.27428048e-02, -3.73441656e-02, -5.37931649e-03,
        -1.88764918e-02, -5.23799248e-03, -4.70317035e-02, -4.72718796e-02,
        -4.56534762e-01, -5.74130614e-03, -1.41201299e-02, -3.78446389e-03,
        -3.86078783e-03, -4.96126211e-03, -6.46517538e-03, -2.61985894e-02,
         0.00000000e+00, -6.21152960e-03, -4.92960681e-02, -1.22022502e+00,
        -9.80244661e-01, -2.53194319e-01, -2.52082136e-01, -4.10242590e-01,
        -4.08484905e-01,  4.96082805e-01, -2.28343440e-01, -2.02418124e-01,
         3.23695338e-01,  5.46387222e-01,  5.33483509e-01, -1.55751398e-01,
        -1.11374173e+00, -1.27643787e-01, -2.54065853e-01, -2.52326165e-01,
        -4.14249404e-01, -4.08841611e-01, -1.06276079e+00, -7.89122833e-01,
         3.26308405e+00, -1.05853808e+00,  1.72094398e+00, -3.90957004e-01,
        -1.65253349e-01, -1.13735258e-01, -1.01311923e-01, -8.48451196e-02,
        -8.41116469e-02, -8.19922384e-02, -5.19454404e-02, -4.19735256e-02,
        -3.2

In [0]:
# Kmeans applied to the standardized dataset
from pyspark.mllib.clustering import KMeans
clusters_std = KMeans.train(data_std, 5, maxIterations = 10, runs = 1, initializationMode = 'random')
clusters_size_std = data.map(lambda e: clusters_std.predict(e)).countByValue()

# show the clusters after optimizing using Kmeans
clusters_size_std

defaultdict(int, {1: 296131, 2: 1, 3: 14897})

In [0]:
# show the distances for each element in the training set
# and show the histogram of distances in each bucket.
distance_data_std = data_std.map(get_distance(clusters_std))
print('distance: \n', distance_data_std.take(10))
distance_data_std.keys().histogram(10)

distance: 
 [(1.3132965274808057, array([-4.39176021e-02, -1.27428048e-02, -3.73441656e-02, -5.37931649e-03,
       -1.88764918e-02, -5.23799248e-03, -4.70317035e-02, -4.72718796e-02,
       -4.56534762e-01, -5.74130614e-03, -1.41201299e-02, -3.78446389e-03,
       -3.86078783e-03, -4.96126211e-03, -6.46517538e-03, -2.61985894e-02,
        0.00000000e+00, -6.21152960e-03, -4.92960681e-02, -1.22022502e+00,
       -9.80244661e-01, -2.53194319e-01, -2.52082136e-01, -4.10242590e-01,
       -4.08484905e-01,  4.96082805e-01, -2.28343440e-01, -2.02418124e-01,
        3.23695338e-01,  5.46387222e-01,  5.33483509e-01, -1.55751398e-01,
       -1.11374173e+00, -1.27643787e-01, -2.54065853e-01, -2.52326165e-01,
       -4.14249404e-01, -4.08841611e-01, -1.06276079e+00, -7.89122833e-01,
        3.26308405e+00, -1.05853808e+00,  1.72094398e+00, -3.90957004e-01,
       -1.65253349e-01, -1.13735258e-01, -1.01311923e-01, -8.48451196e-02,
       -8.41116469e-02, -8.19922384e-02, -5.19454404e-02, -4.19735

([0.03233805044288203,
  67.15455385195263,
  134.27676965346237,
  201.3989854549721,
  268.52120125648185,
  335.6434170579916,
  402.7656328595013,
  469.88784866101105,
  537.0100644625207,
  604.1322802640304,
  671.2544960655403],
 [308276, 2658, 65, 12, 8, 5, 0, 1, 2, 2])

## Conclusion:
- The results show that, there only one observation that is outlier that lie in cluster 1.
- Although, the Clustering is sensitive to data which are having high variance to the others, especially, the data without standardizing. But in this case, after processing data with standardizing, the result is quite better with 1 outliers, instead of more than 1000 outliers without standarding features.